In [1]:
%classpath add jar ../../konduit.jar

In [2]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import org.nd4j.linalg.factory.Nd4j;
import ai.konduit.serving.pipeline.api.data.NDArray;
import ai.konduit.serving.pipeline.util.ObjectMappers;

import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;
    
public class IrisEndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    public IrisEndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter()); }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/json"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data data = Data.empty();
                
                try {
                    data.put("input", NDArray.create(Nd4j.create(new float[] {
                        handler.getBodyAsJson().getFloat("sepal_length"),
                        handler.getBodyAsJson().getFloat("sepal_width"),
                        handler.getBodyAsJson().getFloat("petal_length"),
                        handler.getBodyAsJson().getFloat("petal_width")}, new int[] { 1, 4 })));
                } catch (Exception e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(data);
                
                handler.response().end(ObjectMappers.toJson(exec.getNDArray("output").getAs(float[].class)));
                taskHandler.complete();
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.IrisEndPoint

In [3]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class IrisEndPoints implements HttpEndpoints {

    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new IrisEndPoint(pipelineExecutor));
    }
}

ai.konduit.IrisEndPoints

In [4]:
import java.net.URLClassLoader;
import java.net.URL;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();

for(URL url : urls) {
    System.out.println(new java.io.File(url.toURI()));
}

/tmp/beaker846149715482596187/outDir
/root/konduit/konduit.jar


null

In [7]:
%%bash
java -cp /tmp/beaker846149715482596187/outDir:../../konduit.jar ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id onnx-iris -c onnx.yaml -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker846149715482596187/outDir:../../konduit.jar
INFO: Running command /opt/conda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/onnx-iris.log -Dlogback.configurationFile=/tmp/logback-run_command_867d5cff3c21448a.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c onnx.yaml -Dserving.id=onnx-iris
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs onnx-iris'



In [8]:
%%bash
konduit logs onnx-iris -l 100

03:19:57.347 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/1-pytorch-onnx-iris/onnx.yaml
03:19:57.357 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
03:19:57.359 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
03:19:57.359 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'onnx-iris'
03:19:57.694 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 27 PipelineStepRunnerFactory instances
03:19:58.093 [vert.x-worker-thread-0] INFO  a.k.s.v.verticle.InferenceVerticle - 

####################################################################
#                                                                  #
#    |  /   _ \   \ |  _ \  |  | _ _| __ __|    |  /     |  /      #
#    . <   (   | .  |  |  | |  |   |     |      . <      . <       

In [10]:
%%bash
curl -s -H "Content-Type: application/json" -X POST --data '{"sepal_length":5.1,"sepal_width":3.5,"petal_length":1.4,"petal_width":0.2}' http://localhost:$(konduit inspect onnx-iris -q {port})/infer

[ 0.99312085, 0.0068791825, 6.1220806E-9 ]

